In [1]:
# Setup
import numpy as np
import os
import symforce
symforce.set_symbolic_api("symengine")
symforce.set_log_level("warning")

# Set epsilon to a symbol for safe code generation.  For more information, see the Epsilon tutorial:
# https://symforce.org/tutorials/epsilon_tutorial.html
symforce.set_epsilon_to_symbol()

from symforce import codegen
from symforce.codegen import codegen_util
from symforce import ops
import symforce.symbolic as sf
from symforce.values import Values
from symforce.notebook_util import display, display_code, display_code_file

In [12]:
# Constants
m_c = sf.Symbol('m_c')  # mass of cart
m_p = sf.Symbol('m_p')  # mass of pendulum
l = sf.Symbol('l')  # length of pendulum
g = sf.Symbol('g')  # gravity


# State variables
theta = sf.Symbol('theta')  # angle of pendulum
theta_dot = sf.Symbol('theta_dot')  # angular velocity of pendulum
x = sf.Symbol('x')  # position of cart
x_dot = sf.Symbol('x_dot')  # velocity of cart

q = sf.Matrix([theta, x])
q_dot = sf.Matrix([theta_dot, x_dot])


# Weights
Q = sf.Matrix([[500, 0, 0, 0],
               [0, 100, 0, 0],
               [0, 0, 0, 0],
               [0, 0, 0, 0]])
R = sf.Matrix([[1]])


# Dynamics
H = sf.Matrix([[m_c + m_p, m_p * l * sf.cos(theta)],
              [m_p * l * sf.cos(theta), m_p * l ** 2]])
C = sf.Matrix([[0, -m_p * l * theta_dot * sf.sin(theta)], [0, 0]])
G = sf.Matrix([[0], [m_p * g * l * sf.sin(theta)]])
B_ = sf.Matrix([1, 0])
H_inv = H.inv()
dGdq = G.jacobian(q)

A = sf.Matrix.block_matrix(
    [[sf.Matrix.zeros(2, 2), sf.Matrix.eye(2)],
     [-H_inv * G.jacobian(q), -H_inv * C]])
B = sf.Matrix.block_matrix([[sf.Matrix.zeros(2, 1)],
                            [H_inv * B_]])


# LQR solution
def iterate_K(A: sf.M44, B: sf.M41, R: sf.M11, P: sf.M44) -> sf.M14:
    """Perform one iteration of the LQR algorithm to compute the optimal gain matrix K"""
    return -(R + B.T * P * B).inv() * B.T * P * A

def iterate_P(A: sf.M44, B: sf.M41, Q: sf.M44, R: sf.M11, P: sf.M44, K: sf.M14, K_new: sf.M14) -> sf.M44:
    """Perform one iteration of the LQR algorithm to compute the optimal cost-to-go matrix P"""
    return Q + K_new.T * R * K_new + (A + B * K).T * P * (A + B * K)

In [14]:
"""Generate code for the dynamics"""
dynamics_inputs = Values()

dynamics_inputs["l"] = l

with dynamics_inputs.scope("constants"):
    dynamics_inputs["theta"] = theta
    dynamics_inputs["theta_dot"] = theta_dot
    dynamics_inputs["m_c"] = m_c
    dynamics_inputs["m_p"] = m_p
    dynamics_inputs["g"] = g

dynamics_A_outputs = Values()
dynamics_A_outputs["A"] = A.simplify()
dynamics_B_outputs = Values()
dynamics_B_outputs["B"] = B.simplify()

dynamics_A = codegen.Codegen(
    inputs=dynamics_inputs,
    outputs=dynamics_A_outputs,
    config=codegen.CppConfig(),
    name="dynamics_A",
    return_key="A",
)
dynamics_A_data = dynamics_A.generate_function()

dynamics_B = codegen.Codegen(
    inputs=dynamics_inputs,
    outputs=dynamics_B_outputs,
    config=codegen.CppConfig(),
    name="dynamics_B",
    return_key="B",
)
dynamics_B_data = dynamics_B.generate_function()


"""Generate code for the LQR algorithm"""
iterate_K_codegen = codegen.Codegen.function(
    func=iterate_K,
    config=codegen.CppConfig(),
)
iterate_K_codegen_data = iterate_K_codegen.generate_function()

iterate_P_codegen = codegen.Codegen.function(
    func=iterate_P,
    config=codegen.CppConfig(),
)
iterate_P_codegen_data = iterate_P_codegen.generate_function()

symbolic.simplify():472 WARNING -- Converting to sympy to use .simplify
symbolic.simplify():472 WARNING -- Converting to sympy to use .simplify


Generated 4 files
Generated 4 files


In [22]:
"""Generate Python code"""
namespace = "cartpole_lqr"

dynamics_A_python = codegen.Codegen(
    inputs=dynamics_inputs,
    outputs=dynamics_A_outputs,
    config=codegen.PythonConfig(),
    name="dynamics_A",
    return_key="A",
)
dynamics_A_python_data = dynamics_A_python.generate_function(namespace=namespace)

dynamics_B_python = codegen.Codegen(
    inputs=dynamics_inputs,
    outputs=dynamics_B_outputs,
    config=codegen.PythonConfig(),
    name="dynamics_B",
    return_key="B",
)
dynamics_B_python_data = dynamics_B_python.generate_function(namespace=namespace)

iterate_K_python = codegen.Codegen.function(
    func=iterate_K,
    config=codegen.PythonConfig(),
)
iterate_K_python_data = iterate_K_python.generate_function(namespace=namespace)

iterate_P_python = codegen.Codegen.function(
    func=iterate_P,
    config=codegen.PythonConfig(),
)


# display_code_file(dynamics_A_python_data.generated_files[1], "python")

Generated 4 files
